# Large Gridded Exports from GEE

This notebook shows how to export a country-scale raster from Earth Engine as separate tiles using a grid. The key is to ensure use of `crs` and `crsTransform` to ensure all the tiles are in the same pixel grid and align correctly with the target projection.

[Read the full post](https://spatialthoughts.com/2024/10/23/large-image-exports-gee/)

In [1]:
import geemap
import ee

#### Initialization

First of all, you need to run the following cells to initialize the API and authorize your account. You must have a Google Cloud Project associated with your GEE account. Replace the `cloud_project` with your own project from [Google Cloud Console](https://console.cloud.google.com/).

In [4]:
# Replace the cloud_project with your own project
cloud_project = 'spatialthoughts'
try:
    ee.Initialize(project='ee-zhangjingyievan')
except:
    ee.Authenticate()
    ee.Initialize(project='ee-zhangjingyievan')

#### Data Prep

We select a country and create a clipped ESA WorldCover 2020 classification for the region.

In [5]:
def maskS2clouds(image):
    qa = image.select('QA60')
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
    return image.updateMask(mask).divide(10000)

def getNDVI(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    waterMask = ndwi.lt(0)
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return ndvi.updateMask(waterMask)

country = 'NGA'
geometry = ee.Geometry.Rectangle([3.0982732, 6.393351099999999, 3.564808, 6.7027591])
thresh = 0.17

dataset = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterDate('2019-01-01', '2020-12-31') \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 8)) \
        .map(maskS2clouds) \
        .map(lambda image: image.clip(geometry)) \
        .median()

spectralBands = dataset.select(['B4', 'B3', 'B2', 'B8'])
ndvi = getNDVI(dataset).rename('NDVI')
ndviLabel = ndvi.gt(thresh).rename('NDVI_Label')

In [11]:
vis = {
  'min': 0,
  'max': 1,
  'palette':['white', 'green']
}
m = geemap.Map(width=800)
m.addLayer(ndviLabel, vis, 'NDVI Label');
m.centerObject(geometry, 11)
m

Map(center=[6.548092913864246, 3.3315405999999546], controls=(WidgetControl(options=['position', 'transparent_…

In [14]:
def splitByChunkSize(fc, totalBuildings, chunkSize):

    """ Split buildings into chunks if too large """

    chunkSize = ee.Number(chunkSize)  # Convert to ee.Number
    numChunks = totalBuildings.divide(chunkSize).ceil()  # Total number of chunks

    # Generate start indices for each chunk
    startIndices = ee.List.sequence(0, numChunks.subtract(1)).map(
        lambda i: ee.Number(i).multiply(chunkSize).min(totalBuildings)
    )

    # Split into chunks and convert to FeatureCollections
    chunks = startIndices.map(lambda start:
        ee.FeatureCollection(
            fc.toList(chunkSize.min(totalBuildings.subtract(start)), start)
        )
    )

    return chunks

def isGlobalExtent(feature):

    """ Remove incorrect features """

    bounds = feature.geometry().bounds()
    firstCoordinate = ee.List(bounds.coordinates().get(0))
    minLon = ee.List(firstCoordinate.get(0)).get(0)
    return ee.String(minLon).equals("-Infinity")

def createMask(chunk):

    """ Generate mask for each building chunk """

    chunk = ee.FeatureCollection(chunk)
    mask = chunk.reduceToImage(
        properties=['boundary_id'],
        reducer=ee.Reducer.first()
    ).unmask(0).eq(0).reproject(
        crs=dataset.projection(),
        scale=10
    ).clip(geometry)

    return mask

def maskBuildings(country, geometry, image, scale):

    """ Mask building areas """

    buildings = ee.FeatureCollection( \
        "projects/sat-io/open-datasets/VIDA_COMBINED/" + country) \
        .filterBounds(geometry) \
        .map(lambda feature: feature.set('isGlobal', isGlobalExtent(feature))) \
        .filter(ee.Filter.eq('isGlobal', False))

    buildings = buildings.map(lambda feature: feature.simplify(maxError=5))

    totalBuildings = buildings.size()

    combinedMask = ee.Image(
        ee.Algorithms.If(
            totalBuildings.gt(0),
            # If buildings exist: process chunks and combine masks
            ee.ImageCollection(
                splitByChunkSize(buildings, totalBuildings, 2e5).map(createMask)
            ).reduce(ee.Reducer.min()),
            # If no buildings: return a constant 1 image
            ee.Image.constant(1)
        )
    )

    return combinedMask

In [ ]:
buildingMask = maskBuildings(country, geometry, dataset, 10).rename('building_mask')

#### Create a Grid

We create a grid and calculate the parameters for the CRS Transform. Each tile of the grid will be exported as a separate image on the chosen pixel grid.

In [13]:
# Choose the export CRS
crs = 'EPSG:3857'

# Choose the pixel size for export (meters)
pixelSize = 10

# Choose the export tile size (pixels)
tileSize = 1000

# Calculate the grid size (meters)
gridSize = tileSize * pixelSize

# Create the grid covering the geometry bounds
bounds = geometry.bounds(**{
  'proj': crs, 'maxError': 1
})

grid = bounds.coveringGrid(**{
  'proj':crs, 'scale': gridSize
})

m.addLayer(grid, {'color': 'blue'}, 'Grid')
m

Map(bottom=126440.0, center=[6.757351500676311, 3.3343505859375004], controls=(WidgetControl(options=['positio…

#### Calculate the CRS Transform

In [ ]:
# Calculate the coordinates of the top-left corner of the grid
bounds = grid.geometry().bounds(**{
  'proj': crs, 'maxError': 1
});

# Extract the coordinates of the grid
coordList = ee.Array.cat(bounds.coordinates(), 1)

xCoords = coordList.slice(1, 0, 1)
yCoords = coordList.slice(1, 1, 2)

# We need the coordinates of the top-left pixel
xMin = xCoords.reduce('min', [0]).get([0,0])
yMax = yCoords.reduce('max', [0]).get([0,0])

# Create the CRS Transform

# The transform consists of 6 parameters:
# [xScale, xShearing, xTranslation,
#  yShearing, yScale, yTranslation]
transform = ee.List([
    pixelSize, 0, xMin, 0, -pixelSize, yMax]).getInfo()
print(transform)

#### Resample or Aggregate Pixels

By default, the images are resampled to the target pixel grid using the Nearest Neighbor method. This is fine for most types of images, but you may want to change this behavior for certain types of operations. For discrete rasters, such as landcover classification, nearest neighbor is appropriate. For climate or elevation rasters, you may want to enable `bilinear` or `bicubic` interpolation.



In [ ]:
# Uncomment below to enable resampling
# This is not required for classification images
# image = image.resample('bicubic')

#### Set a NoData Value

This is an important step. If you have masked pixels in your image, the output tiles will not be of equal size. To ensure each tile has the same dimensions and there are no gaps or overlapping pixels, `unmask()` all masked pixels and set a nodata value.

In [ ]:
# Assign a no-data value
noDataValue = 0
exportImage = image.unmask(**{
    'value':noDataValue,
    'sameFootprint': False
})

#### Export Tiles

In [ ]:
tile_ids = grid.aggregate_array('system:index').getInfo();
print('Total tiles', len(tile_ids))

Total tiles 20


In [ ]:
# Export each tile
# Warning: This will result in 20 large GeoTIFFs tiles in your Google Drive
for i, tile_id in enumerate(tile_ids):
    feature = ee.Feature(grid.toList(1, i).get(0))
    geometry = feature.geometry()
    task_name = 'tile_' + tile_id.replace(',', '_')
    task = ee.batch.Export.image.toDrive(**{
        'image': exportImage,
        'description': f'Image_Export_{task_name}',
        'fileNamePrefix': task_name,
        'folder':'earthengine',
        'crs': crs,
        'crsTransform': transform,
        'region': geometry,
        'maxPixels': 1e10
    })
    task.start()
    print('Started Task: ', i+1)